<a href="https://colab.research.google.com/github/Sudin2001/Project-Of-ML/blob/main/Financial_Risk_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CASH FLOW FORECASTING MODEL FOR LIQUIDITY RISK ANALYSIS
# A Comprehensive Python Implementation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Statistical and ML libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

print("="*10)
print("CASH FLOW FORECASTING MODEL FOR LIQUIDITY RISK ANALYSIS")
print("TechManuf Industries Ltd - Financial Risk Management Project")
print("="*10)
print("\n PROJECT OVERVIEW:")
print("• Objective: Build predictive models for cash flow forecasting")
print("• Risk Focus: Liquidity risk assessment and early warning system")
print("• Data Period: January 2021 - December 2024 (4 years)")
print("• Methodology: Time series analysis, regression modeling, stress testing")


CASH FLOW FORECASTING MODEL FOR LIQUIDITY RISK ANALYSIS
TechManuf Industries Ltd - Financial Risk Management Project

 PROJECT OVERVIEW:
• Objective: Build predictive models for cash flow forecasting
• Risk Focus: Liquidity risk assessment and early warning system
• Data Period: January 2021 - December 2024 (4 years)
• Methodology: Time series analysis, regression modeling, stress testing


In [ ]:

print("\n DATA LOADING AND EXPLORATION")
print("-" * 50)

# Main weekly cash flows
weekly_df = pd.read_csv('/content/drive/MyDrive/Financial Risk Management/weekly_cash_flows.csv')
weekly_df['Date'] = pd.to_datetime(weekly_df['Date'])

# Monthly aggregated data
monthly_df = pd.read_csv('/content/drive/MyDrive/Financial Risk Management/monthly_cash_flows.csv')
monthly_df['Date'] = pd.to_datetime(monthly_df['Date'])

# Economic indicators
economic_df = pd.read_csv('/content/drive/MyDrive/Financial Risk Management/economic_indicators.csv')
economic_df['Date'] = pd.to_datetime(economic_df['Date'])

# Accounts receivable aging
ar_df = pd.read_csv('/content/drive/MyDrive/Financial Risk Management/accounts_receivable_aging.csv')
ar_df['Date'] = pd.to_datetime(ar_df['Date'])

# Supplier payment terms
supplier_df = pd.read_csv('/content/drive/MyDrive/Financial Risk Management/supplier_payment_terms.csv')
supplier_df['Date'] = pd.to_datetime(supplier_df['Date'])

# Dataset summaries
datasets = {
    'Weekly Cash Flows': weekly_df,
    'Monthly Cash Flows': monthly_df,
    'Economic Indicators': economic_df,
    'AR Aging Analysis': ar_df,
    'Supplier Payments': supplier_df
}

for name, df in datasets.items():
    print(f"  {name}:")
    print(f"   Shape: {df.shape}")
    print(f"   Period: {df['Date'].min().date()} to {df['Date'].max().date()}")
    print(f"   Columns: {len(df.columns)}")
    print()

print("  Primary focus will be on Weekly Cash Flows for detailed modeling")
print(f"   {len(weekly_df)} weeks of comprehensive cash flow data")
print(f"   {weekly_df.columns.tolist()}")
print(weekly_df.head())


 DATA LOADING AND EXPLORATION
--------------------------------------------------
  Weekly Cash Flows:
   Shape: (209, 21)
   Period: 2021-01-04 to 2024-12-30
   Columns: 21

  Monthly Cash Flows:
   Shape: (48, 19)
   Period: 2021-01-01 to 2024-12-01
   Columns: 19

  Economic Indicators:
   Shape: (48, 9)
   Period: 2021-01-31 to 2024-12-31
   Columns: 9

  AR Aging Analysis:
   Shape: (209, 11)
   Period: 2021-01-04 to 2024-12-30
   Columns: 11

  Supplier Payments:
   Shape: (1254, 8)
   Period: 2021-01-04 to 2024-12-30
   Columns: 8

  Primary focus will be on Weekly Cash Flows for detailed modeling
   209 weeks of comprehensive cash flow data
   ['Date', 'Week', 'Month', 'Quarter', 'Year', 'Customer_Receipts', 'Interest_Income', 'Other_Income', 'Total_Inflows', 'Supplier_Payments', 'Payroll', 'Rent_Utilities', 'Tax_Payments', 'Interest_Expense', 'Capex', 'Other_Expenses', 'Total_Outflows', 'Net_Cash_Flow', 'Cumulative_Cash_Flow', 'Cash_Position', 'Emergency_Financing']
        Da

In [ ]:
print("\n EXPLORATORY DATA ANALYSIS")
print("-" * 50)

# Key statistics for main cash flow metrics
print(" CASH FLOW SUMMARY STATISTICS (₹000s)")
print("=" * 60)

key_metrics = ['Total_Inflows', 'Total_Outflows', 'Net_Cash_Flow', 'Cash_Position']
summary_stats = weekly_df[key_metrics].describe()
print(summary_stats.round(2))

print(f" Average weekly net cash flow: ₹{weekly_df['Net_Cash_Flow'].mean():.0f}K")
print(f" Cash flow volatility (std dev): ₹{weekly_df['Net_Cash_Flow'].std():.0f}K")
print(f" Minimum cash position: ₹{weekly_df['Cash_Position'].min():.0f}K")
print(f" Final cash position: ₹{weekly_df['Cash_Position'].iloc[-1]:.0f}K")
print(f" Cash position growth: {((weekly_df['Cash_Position'].iloc[-1] / weekly_df['Cash_Position'].iloc[0]) - 1) * 100:.1f}%")

# Seasonal patterns analysis
print("\n SEASONAL ANALYSIS:")

# Group by quarter to see seasonal patterns
quarterly_analysis = weekly_df.groupby('Quarter').agg({
    'Net_Cash_Flow': ['mean', 'std'],
    'Total_Inflows': 'mean',
    'Total_Outflows': 'mean'
}).round(2)

quarterly_analysis.columns = ['Net_Flow_Mean', 'Net_Flow_Std', 'Avg_Inflows', 'Avg_Outflows']
print("Quarterly Cash Flow Patterns:")
print(quarterly_analysis)

print(f"\n Best performing quarter: Q{quarterly_analysis['Net_Flow_Mean'].idxmax()} (₹{quarterly_analysis['Net_Flow_Mean'].max():.0f}K avg)")
print(f" Weakest performing quarter: Q{quarterly_analysis['Net_Flow_Mean'].idxmin()} (₹{quarterly_analysis['Net_Flow_Mean'].min():.0f}K avg)")

# Year-over-year growth analysis
print("\n YEAR-OVER-YEAR ANALYSIS:")

yearly_analysis = weekly_df.groupby('Year').agg({
    'Net_Cash_Flow': ['sum', 'mean'],
    'Total_Inflows': 'sum',
    'Total_Outflows': 'sum'
}).round(2)

yearly_analysis.columns = ['Annual_Net_Flow', 'Weekly_Avg_Net', 'Annual_Inflows', 'Annual_Outflows']
print("Annual Performance:")
print(yearly_analysis)

# Calculate growth rates
for year in range(2022, 2025):
    prev_year = year - 1
    if prev_year in yearly_analysis.index and year in yearly_analysis.index:
        growth = ((yearly_analysis.loc[year, 'Annual_Inflows'] / yearly_analysis.loc[prev_year, 'Annual_Inflows']) - 1) * 100
        print(f"Revenue growth {prev_year}-{year}: {growth:.1f}%")

print(f"\nOverall average annual growth: {((yearly_analysis['Annual_Inflows'].iloc[-1] / yearly_analysis['Annual_Inflows'].iloc[0]) ** (1/3) - 1) * 100:.1f}%")


 EXPLORATORY DATA ANALYSIS
--------------------------------------------------
 CASH FLOW SUMMARY STATISTICS (₹000s)
       Total_Inflows  Total_Outflows  Net_Cash_Flow  Cash_Position
count         209.00          209.00         209.00         209.00
mean         1074.78          735.47         339.32       39380.24
std           309.86          239.51         259.67       20867.92
min           401.29          232.39        -276.37        5485.62
25%           847.90          519.90         158.15       20174.38
50%          1073.46          748.61         323.30       38906.61
75%          1286.18          933.21         526.47       57288.28
max          1940.26         1261.53        1070.05       75917.12
 Average weekly net cash flow: ₹339K
 Cash flow volatility (std dev): ₹260K
 Minimum cash position: ₹5486K
 Final cash position: ₹75917K
 Cash position growth: 1283.9%

 SEASONAL ANALYSIS:
Quarterly Cash Flow Patterns:
         Net_Flow_Mean  Net_Flow_Std  Avg_Inflows  Avg_Outflo

In [ ]:


print("\n LIQUIDITY RISK ANALYSIS")
print("-" * 50)

# Merge AR aging data with weekly cash flows for comprehensive analysis
risk_df = weekly_df.merge(ar_df[['Date', 'Total_AR', 'Collection_Risk_Score']], on='Date', how='left')

# AR risk statistics
ar_stats = ar_df[['Total_AR', 'Current_0_30_Days', 'Days_31_60', 'Days_61_90', 'Days_Over_90', 'Collection_Risk_Score']].describe()
print(ar_stats.round(2))

print(f"\n COLLECTION RISK INSIGHTS:")
print(f" Average collection risk score: {ar_df['Collection_Risk_Score'].mean():.2f}%")
print(f" Maximum collection risk: {ar_df['Collection_Risk_Score'].max():.2f}%")
print(f" Percentage of weeks with risk > 3%: {(ar_df['Collection_Risk_Score'] > 3).sum() / len(ar_df) * 100:.1f}%")

# Calculate Days Sales Outstanding (DSO)
risk_df['DSO'] = risk_df['Total_AR'] / (risk_df['Customer_Receipts'] / 7)  # Convert weekly receipts to daily
print(f" Average Days Sales Outstanding (DSO): {risk_df['DSO'].mean():.1f} days")

# Cash conversion cycle analysis
print(f"\n CASH CONVERSION METRICS:")
print("-" * 30)

# Calculate working capital components
risk_df['AR_to_Sales_Ratio'] = risk_df['Total_AR'] / risk_df['Customer_Receipts']
risk_df['Cash_Flow_to_AR_Ratio'] = risk_df['Net_Cash_Flow'] / risk_df['Total_AR']

print(f" AR as % of weekly sales: {risk_df['AR_to_Sales_Ratio'].mean():.1f}x")
print(f" Net cash flow as % of AR: {risk_df['Cash_Flow_to_AR_Ratio'].mean():.2f}%")

# Liquidity ratios and warning indicators
print(f"\n  LIQUIDITY WARNING INDICATORS:")
print("-" * 40)

# Define liquidity thresholds
min_cash_threshold = 2000  # Minimum safe cash level
critical_risk_threshold = 3.5  # Critical collection risk %

# Calculate warning indicators
low_cash_weeks = (risk_df['Cash_Position'] < min_cash_threshold).sum()
high_risk_weeks = (risk_df['Collection_Risk_Score'] > critical_risk_threshold).sum()
negative_flow_weeks = (risk_df['Net_Cash_Flow'] < 0).sum()

print(f" Weeks with cash below ₹{min_cash_threshold}K threshold: {low_cash_weeks} ({low_cash_weeks/len(risk_df)*100:.1f}%)")
print(f" Weeks with collection risk > {critical_risk_threshold}%: {high_risk_weeks} ({high_risk_weeks/len(risk_df)*100:.1f}%)")
print(f" Weeks with negative cash flow: {negative_flow_weeks} ({negative_flow_weeks/len(risk_df)*100:.1f}%)")

# Merge with economic data (monthly to weekly approximation)
economic_monthly = economic_df.set_index('Date')
risk_monthly = risk_df.groupby([risk_df['Date'].dt.to_period('M')]).agg({
    'Net_Cash_Flow': 'sum',
    'Customer_Receipts': 'sum',
    'Collection_Risk_Score': 'mean'
}).reset_index()

risk_monthly['Date'] = risk_monthly['Date'].dt.to_timestamp()
economic_merged = economic_monthly.merge(risk_monthly, left_index=True, right_on='Date', how='inner')

# Calculate correlations
correlations = {
    'GDP Growth vs Net Cash Flow': np.corrcoef(economic_merged['GDP_Growth_Rate'], economic_merged['Net_Cash_Flow'])[0,1],
    'Inflation vs Customer Receipts': np.corrcoef(economic_merged['Inflation_Rate'], economic_merged['Customer_Receipts'])[0,1],
    'Interest Rate vs Collection Risk': np.corrcoef(economic_merged['Interest_Rate'], economic_merged['Collection_Risk_Score'])[0,1],
    'USD/INR vs Cash Flow': np.corrcoef(economic_merged['USD_INR_Exchange_Rate'], economic_merged['Net_Cash_Flow'])[0,1]
}

for factor, corr in correlations.items():
    print(f" {factor}: {corr:.3f}")

print("\n Risk analysis complete - Moving to time series modeling...")


 LIQUIDITY RISK ANALYSIS
--------------------------------------------------
       Total_AR  Current_0_30_Days  Days_31_60  Days_61_90  Days_Over_90  \
count    209.00             209.00      209.00      209.00        209.00   
mean    1930.74            1287.81      382.86      164.86         95.21   
std      557.70             380.22      117.33       63.64         43.15   
min      718.96             482.41      136.83       47.32         25.08   
25%     1523.29            1008.73      287.32      120.61         60.38   
50%     1927.96            1272.96      364.86      155.20         88.46   
75%     2311.04            1551.70      460.71      205.30        126.49   
max     3488.35            2324.62      717.85      398.18        260.46   

       Collection_Risk_Score  
count                 209.00  
mean                    2.33  
std                     0.52  
min                     1.14  
25%                     1.93  
50%                     2.36  
75%                  

In [ ]:

print(" ECONOMIC CORRELATION ANALYSIS (CORRECTED):")
print("-" * 50)

# Method 1: Use existing monthly_df (cleanest approach)
try:
    # Use the pre-aggregated monthly data that was already loaded
    economic_clean = economic_df[['Date', 'GDP_Growth_Rate', 'Inflation_Rate',
                                  'Interest_Rate', 'USD_INR_Exchange_Rate',
                                  'Manufacturing_PMI']].copy()

    monthly_clean = monthly_df[['Date', 'Net_Cash_Flow', 'Customer_Receipts',
                               'Total_Inflows']].copy()

    # Ensure dates are in the same format
    economic_clean['Date'] = pd.to_datetime(economic_clean['Date'])
    monthly_clean['Date'] = pd.to_datetime(monthly_clean['Date'])

    # Merge the datasets
    economic_analysis = economic_clean.merge(monthly_clean, on='Date', how='inner')

    print(f" Successfully merged {len(economic_analysis)} months of data")

except Exception as e:
    print(f" Using alternative approach due to: {str(e)}")

    # Method 2: Manual aggregation without reset_index conflicts
    monthly_aggregated = weekly_df.copy()
    monthly_aggregated['YearMonth'] = monthly_aggregated['Date'].dt.to_period('M')

    monthly_summary = monthly_aggregated.groupby('YearMonth').agg({
        'Net_Cash_Flow': 'sum',
        'Customer_Receipts': 'sum',
        'Total_Inflows': 'sum'
    })

    # Create proper date column
    monthly_summary['Date'] = monthly_summary.index.to_timestamp()
    monthly_summary = monthly_summary.reset_index(drop=True)

    # Merge with economic data
    economic_analysis = economic_df.merge(monthly_summary, on='Date', how='inner')

    print(f" Successfully merged {len(economic_analysis)} months of data (alternative method)")

# Calculate correlations (this part remains the same)
if len(economic_analysis) > 0:
    correlations = {
        'GDP Growth vs Net Cash Flow': economic_analysis['GDP_Growth_Rate'].corr(economic_analysis['Net_Cash_Flow']),
        'Inflation vs Customer Receipts': economic_analysis['Inflation_Rate'].corr(economic_analysis['Customer_Receipts']),
        'Interest Rate vs Net Cash Flow': economic_analysis['Interest_Rate'].corr(economic_analysis['Net_Cash_Flow']),
        'USD/INR vs Cash Flow': economic_analysis['USD_INR_Exchange_Rate'].corr(economic_analysis['Net_Cash_Flow']),
        'Manufacturing PMI vs Inflows': economic_analysis['Manufacturing_PMI'].corr(economic_analysis['Total_Inflows'])
    }

    print("\n ECONOMIC CORRELATIONS WITH CASH FLOWS:")
    for factor, corr in correlations.items():
        if not pd.isna(corr):
            strength = "Strong" if abs(corr) > 0.5 else "Moderate" if abs(corr) > 0.3 else "Weak"
            direction = "Positive" if corr > 0 else "Negative"
            print(f"• {factor}: {corr:.3f} ({strength} {direction})")
        else:
            print(f"• {factor}: No correlation data available")
else:
    print(" No overlapping data found for correlation analysis")

print(f"\n Economic correlation analysis completed successfully")


 ECONOMIC CORRELATION ANALYSIS (CORRECTED):
--------------------------------------------------
 Successfully merged 0 months of data
 No overlapping data found for correlation analysis

 Economic correlation analysis completed successfully


In [ ]:
# Let me fix the correlation analysis with a simpler approach
print(" ECONOMIC CORRELATION ANALYSIS (SIMPLIFIED):")
print("-" * 50)

# Use the existing monthly_df that was already loaded
economic_clean = economic_df[['Date', 'GDP_Growth_Rate', 'Inflation_Rate', 'Interest_Rate', 'USD_INR_Exchange_Rate', 'Manufacturing_PMI']].copy()
monthly_clean = monthly_df[['Date', 'Net_Cash_Flow', 'Customer_Receipts', 'Total_Inflows']].copy()

# Merge properly
economic_analysis = economic_clean.merge(monthly_clean, on='Date', how='inner')
print(f"Successfully merged {len(economic_analysis)} months of data")

# Calculate correlations
correlations = {
    'GDP Growth vs Net Cash Flow': economic_analysis['GDP_Growth_Rate'].corr(economic_analysis['Net_Cash_Flow']),
    'Inflation vs Customer Receipts': economic_analysis['Inflation_Rate'].corr(economic_analysis['Customer_Receipts']),
    'Interest Rate vs Net Cash Flow': economic_analysis['Interest_Rate'].corr(economic_analysis['Net_Cash_Flow']),
    'USD/INR vs Cash Flow': economic_analysis['USD_INR_Exchange_Rate'].corr(economic_analysis['Net_Cash_Flow']),
    'Manufacturing PMI vs Inflows': economic_analysis['Manufacturing_PMI'].corr(economic_analysis['Total_Inflows'])
}

print("\n ECONOMIC CORRELATIONS WITH CASH FLOWS:")
for factor, corr in correlations.items():
    if not pd.isna(corr):
        strength = "Strong" if abs(corr) > 0.5 else "Moderate" if abs(corr) > 0.3 else "Weak"
        direction = "Positive" if corr > 0 else "Negative"
        print(f" {factor}: {corr:.3f} ({strength} {direction})")

print(" PHASE 4: TIME SERIES ANALYSIS & FORECASTING")

# Prepare data for time series analysis
ts_data = weekly_df[['Date', 'Net_Cash_Flow', 'Total_Inflows', 'Total_Outflows', 'Cash_Position']].copy()
ts_data.set_index('Date', inplace=True)

print("\n TIME SERIES CHARACTERISTICS:")

# Check for stationarity
def check_stationarity(timeseries, title):
    # Perform Augmented Dickey-Fuller test
    result = adfuller(timeseries.dropna())
    print(f'\n{title}:')
    print(f'  ADF Statistic: {result[0]:.6f}')
    print(f'  p-value: {result[1]:.6f}')
    print(f'  Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')

    if result[1] <= 0.05:
        print("  Result: Series is stationary")
        return True
    else:
        print("  Result: Series is non-stationary")
        return False

# Check stationarity for key metrics
stationarity_results = {}
for col in ['Net_Cash_Flow', 'Total_Inflows', 'Total_Outflows']:
    stationarity_results[col] = check_stationarity(ts_data[col], col)

print(f"\n Stationarity Analysis Complete")
print(f" Stationary series: {sum(stationarity_results.values())} out of {len(stationarity_results)}")

 ECONOMIC CORRELATION ANALYSIS (SIMPLIFIED):
--------------------------------------------------
Successfully merged 0 months of data

 ECONOMIC CORRELATIONS WITH CASH FLOWS:
 PHASE 4: TIME SERIES ANALYSIS & FORECASTING

 TIME SERIES CHARACTERISTICS:

Net_Cash_Flow:
  ADF Statistic: -3.178344
  p-value: 0.021273
  Critical Values:
	1%: -3.464
	5%: -2.876
	10%: -2.575
  Result: Series is stationary

Total_Inflows:
  ADF Statistic: -3.035918
  p-value: 0.031674
  Critical Values:
	1%: -3.463
	5%: -2.876
	10%: -2.574
  Result: Series is stationary

Total_Outflows:
  ADF Statistic: -3.372931
  p-value: 0.011917
  Critical Values:
	1%: -3.464
	5%: -2.876
	10%: -2.575
  Result: Series is stationary

 Stationarity Analysis Complete
 Stationary series: 3 out of 3


In [ ]:


print("\n SEASONAL DECOMPOSITION ANALYSIS:")
print("-" * 50)

# Perform seasonal decomposition for Net Cash Flow
print("Decomposing Net Cash Flow time series...")

# For weekly data, we use 52 weeks as the seasonal period
decomposition = seasonal_decompose(ts_data['Net_Cash_Flow'], model='additive', period=52)

# Extract components
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

print(f" Decomposition complete")
print(f" Trend component range: {trend.min():.0f} to {trend.max():.0f}")
print(f" Seasonal component range: {seasonal.min():.0f} to {seasonal.max():.0f}")
print(f" Residual std deviation: {residual.std():.0f}")

# Analyze seasonal patterns
seasonal_profile = seasonal.groupby(seasonal.index.isocalendar().week).mean()
peak_weeks = seasonal_profile.nlargest(5)
low_weeks = seasonal_profile.nsmallest(5)

print(f"\n SEASONAL INSIGHTS:")
print(f" Peak season weeks (highest cash flow):")
for week, value in peak_weeks.items():
    print(f"  Week {week}: +₹{value:.0f}K above trend")

print(f" Low season weeks (lowest cash flow):")
for week, value in low_weeks.items():
    print(f"  Week {week}: ₹{value:.0f}K below trend")


print(f"\n BUILDING FORECASTING MODELS:")
print("-" * 50)

# Split data for train/test
train_size = int(len(ts_data) * 0.8)  # 80% for training
train_data = ts_data[:train_size]
test_data = ts_data[train_size:]

print(f" Training period: {train_data.index[0].date()} to {train_data.index[-1].date()} ({len(train_data)} weeks)")
print(f" Testing period: {test_data.index[0].date()} to {test_data.index[-1].date()} ({len(test_data)} weeks)")

# Model 1: ARIMA Model
print(f"\n MODEL 1: ARIMA FORECASTING")

try:
    # Fit ARIMA model (using auto selection approach)
    # Start with ARIMA(1,1,1) as baseline
    arima_model = ARIMA(train_data['Net_Cash_Flow'], order=(1,1,1))
    arima_fitted = arima_model.fit()

    print(f" ARIMA(1,1,1) model fitted successfully")
    print(f" AIC: {arima_fitted.aic:.2f}")
    print(f" BIC: {arima_fitted.bic:.2f}")

    # Make predictions
    arima_forecast = arima_fitted.forecast(steps=len(test_data))
    arima_mae = mean_absolute_error(test_data['Net_Cash_Flow'], arima_forecast)
    arima_mse = mean_squared_error(test_data['Net_Cash_Flow'], arima_forecast)

    print(f" Mean Absolute Error: ₹{arima_mae:.0f}K")
    print(f" Root Mean Squared Error: ₹{np.sqrt(arima_mse):.0f}K")

except Exception as e:
    print(f" ARIMA model failed: {str(e)}")
    arima_forecast = None

# Model 2: Linear Regression with features
print(f"\n MODEL 2: REGRESSION WITH ECONOMIC FACTORS")
print("-" * 30)

# Prepare features for regression
train_features = train_data.copy()
test_features = test_data.copy()

# Add lag features
for lag in [1, 2, 4]:  # 1, 2, and 4 week lags
    train_features[f'Net_Flow_lag_{lag}'] = train_features['Net_Cash_Flow'].shift(lag)
    test_features[f'Net_Flow_lag_{lag}'] = test_features['Net_Cash_Flow'].shift(lag)

# Add seasonal features
train_features['Week_of_Year'] = train_features.index.isocalendar().week
train_features['Month'] = train_features.index.month
train_features['Quarter'] = train_features.index.quarter

test_features['Week_of_Year'] = test_features.index.isocalendar().week
test_features['Month'] = test_features.index.month
test_features['Quarter'] = test_features.index.quarter

# Add trend feature
train_features['Trend'] = range(len(train_features))
test_features['Trend'] = range(len(train_features), len(train_features) + len(test_features))

# Select features for modeling
feature_cols = ['Net_Flow_lag_1', 'Net_Flow_lag_2', 'Net_Flow_lag_4', 'Week_of_Year', 'Month', 'Quarter', 'Trend']
X_train = train_features[feature_cols].dropna()
y_train = train_features['Net_Cash_Flow'][X_train.index]

X_test = test_features[feature_cols].dropna()
y_test = test_features['Net_Cash_Flow'][X_test.index]

print(f" Training samples: {len(X_train)}")
print(f" Test samples: {len(X_test)}")

# Fit regression model
reg_model = RandomForestRegressor(n_estimators=100, random_state=42)
reg_model.fit(X_train, y_train)

# Make predictions
reg_forecast = reg_model.predict(X_test)
reg_mae = mean_absolute_error(y_test, reg_forecast)
reg_mse = mean_squared_error(y_test, reg_forecast)

print(f" Random Forest model fitted successfully")
print(f" Mean Absolute Error: ₹{reg_mae:.0f}K")
print(f" Root Mean Squared Error: ₹{np.sqrt(reg_mse):.0f}K")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': reg_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n Most Important Features:")
for idx, row in feature_importance.head(3).iterrows():
    print(f" {row['Feature']}: {row['Importance']:.3f}")

print(f"\n Forecasting Models Complete - Proceeding to stress testing...")


 SEASONAL DECOMPOSITION ANALYSIS:
--------------------------------------------------
Decomposing Net Cash Flow time series...
 Decomposition complete
 Trend component range: 282 to 387
 Seasonal component range: -557 to 456
 Residual std deviation: 132

 SEASONAL INSIGHTS:
 Peak season weeks (highest cash flow):
  Week 5: +₹456K above trend
  Week 3: +₹397K above trend
  Week 47: +₹381K above trend
  Week 43: +₹306K above trend
  Week 19: +₹290K above trend
 Low season weeks (lowest cash flow):
  Week 26: ₹-557K below trend
  Week 34: ₹-377K below trend
  Week 32: ₹-319K below trend
  Week 24: ₹-318K below trend
  Week 12: ₹-304K below trend

 BUILDING FORECASTING MODELS:
--------------------------------------------------
 Training period: 2021-01-04 to 2024-03-11 (167 weeks)
 Testing period: 2024-03-18 to 2024-12-30 (42 weeks)

 MODEL 1: ARIMA FORECASTING
 ARIMA(1,1,1) model fitted successfully
 AIC: 2273.45
 BIC: 2282.78
 Mean Absolute Error: ₹277K
 Root Mean Squared Error: ₹332K

 

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)


 Random Forest model fitted successfully
 Mean Absolute Error: ₹182K
 Root Mean Squared Error: ₹215K

 Most Important Features:
 Net_Flow_lag_2: 0.343
 Net_Flow_lag_4: 0.323
 Net_Flow_lag_1: 0.147

 Forecasting Models Complete - Proceeding to stress testing...


In [ ]:

print("\n PHASE 5: STRESS TESTING & SCENARIO ANALYSIS")
print("="*80)

# Define stress scenarios
stress_scenarios = {
    'Base Case': {
        'revenue_shock': 0.0,
        'collection_delay': 0.0,
        'cost_increase': 0.0,
        'description': 'Normal operating conditions'
    },
    'Mild Recession': {
        'revenue_shock': -0.10,  # 10% revenue decline
        'collection_delay': 0.15,  # 15% increase in collection time
        'cost_increase': 0.05,  # 5% cost increase
        'description': 'Economic slowdown scenario'
    },
    'Severe Recession': {
        'revenue_shock': -0.25,  # 25% revenue decline
        'collection_delay': 0.30,  # 30% increase in collection time
        'cost_increase': 0.10,  # 10% cost increase
        'description': 'Severe economic downturn'
    },
    'Supply Chain Crisis': {
        'revenue_shock': -0.05,  # 5% revenue decline
        'collection_delay': 0.10,  # 10% collection delay
        'cost_increase': 0.20,  # 20% cost increase
        'description': 'Supply chain disruption scenario'
    },
    'Customer Credit Crisis': {
        'revenue_shock': -0.15,  # 15% revenue decline
        'collection_delay': 0.50,  # 50% increase in collection time
        'cost_increase': 0.02,  # 2% cost increase
        'description': 'Major customer payment issues'
    }
}

print(" RUNNING STRESS SCENARIOS:")

# Use the last 13 weeks as baseline for projection
baseline_data = weekly_df.tail(13).copy()
baseline_avg_inflow = baseline_data['Total_Inflows'].mean()
baseline_avg_outflow = baseline_data['Total_Outflows'].mean()
current_cash_position = weekly_df['Cash_Position'].iloc[-1]

print(f" Baseline Metrics (Last 13 weeks):")
print(f"• Average weekly inflows: ₹{baseline_avg_inflow:.0f}K")
print(f"• Average weekly outflows: ₹{baseline_avg_outflow:.0f}K")
print(f"• Current cash position: ₹{current_cash_position:.0f}K")

# Run stress scenarios
scenario_results = {}
projection_weeks = 26  # 6 month projection

print(f"\n STRESS TEST RESULTS ({projection_weeks}-week projection):")
for scenario_name, scenario in stress_scenarios.items():
    print(f"\n {scenario_name}: {scenario['description']}")

    # Apply stress factors
    stressed_inflows = baseline_avg_inflow * (1 + scenario['revenue_shock'])
    stressed_outflows = baseline_avg_outflow * (1 + scenario['cost_increase'])

    # Account for collection delays (reduced cash inflows)
    collection_impact = baseline_avg_inflow * scenario['collection_delay']
    effective_inflows = stressed_inflows - collection_impact

    # Project cash flows
    weekly_net_flow = effective_inflows - stressed_outflows

    # Calculate cumulative impact
    projected_cash_position = current_cash_position + (weekly_net_flow * projection_weeks)

    # Calculate weeks until liquidity crisis (cash < 1000K)
    crisis_threshold = 1000
    weeks_to_crisis = None
    if weekly_net_flow < 0:
        weeks_to_crisis = (current_cash_position - crisis_threshold) / abs(weekly_net_flow)
        weeks_to_crisis = max(0, weeks_to_crisis)

    scenario_results[scenario_name] = {
        'weekly_net_flow': weekly_net_flow,
        'projected_cash_position': projected_cash_position,
        'weeks_to_crisis': weeks_to_crisis,
        'total_cash_impact': weekly_net_flow * projection_weeks
    }

    print(f"    Weekly net cash flow: ₹{weekly_net_flow:.0f}K")
    print(f"    Projected cash position: ₹{projected_cash_position:.0f}K")
    if weeks_to_crisis and weeks_to_crisis < projection_weeks:
        print(f"     Liquidity crisis in: {weeks_to_crisis:.1f} weeks")
    else:
        print(f"    No liquidity crisis expected")
    print(f"    Total 26-week impact: ₹{weekly_net_flow * projection_weeks:.0f}K")

# Risk ranking
print(f"\n RISK RANKING (Most Critical Scenarios):")

risk_ranking = sorted(
    [(name, results['weeks_to_crisis']) for name, results in scenario_results.items()
     if results['weeks_to_crisis'] is not None],
    key=lambda x: x[1] if x[1] is not None else float('inf')
)

for i, (scenario, weeks) in enumerate(risk_ranking[:3], 1):
    print(f"{i}. {scenario}: Liquidity crisis in {weeks:.1f} weeks")

if not risk_ranking:
    print(" No liquidity crises projected in any scenario")

# Monte Carlo simulation for uncertainty
print(f"\n MONTE CARLO UNCERTAINTY ANALYSIS:")

np.random.seed(42)
num_simulations = 1000

# Use historical volatility for simulation
historical_volatility = weekly_df['Net_Cash_Flow'].std()
print(f" Historical volatility: ₹{historical_volatility:.0f}K")
print(f" Running {num_simulations:,} simulations...")

# Monte Carlo for base case scenario
base_weekly_flow = scenario_results['Base Case']['weekly_net_flow']
simulated_outcomes = []

for _ in range(num_simulations):
    random_shocks = np.random.normal(0, historical_volatility, projection_weeks)
    simulated_flows = base_weekly_flow + random_shocks
    final_position = current_cash_position + simulated_flows.sum()
    simulated_outcomes.append(final_position)

simulated_outcomes = np.array(simulated_outcomes)

# Calculate confidence intervals
confidence_levels = [0.05, 0.25, 0.75, 0.95]
percentiles = np.percentile(simulated_outcomes, [5, 25, 75, 95])

print(f"\n MONTE CARLO CONFIDENCE INTERVALS:")
print(f" 5th percentile (Worst 5%): ₹{percentiles[0]:.0f}K")
print(f" 25th percentile: ₹{percentiles[1]:.0f}K")
print(f" 75th percentile: ₹{percentiles[2]:.0f}K")
print(f" 95th percentile (Best 5%): ₹{percentiles[3]:.0f}K")

# Risk metrics
crisis_probability = (simulated_outcomes < crisis_threshold).mean() * 100
print(f" Probability of liquidity crisis: {crisis_probability:.1f}%")

print(f"\n Stress testing complete - Creating final dashboard...")


 PHASE 5: STRESS TESTING & SCENARIO ANALYSIS
 RUNNING STRESS SCENARIOS:
 Baseline Metrics (Last 13 weeks):
• Average weekly inflows: ₹1301K
• Average weekly outflows: ₹849K
• Current cash position: ₹75917K

 STRESS TEST RESULTS (26-week projection):

 Base Case: Normal operating conditions
    Weekly net cash flow: ₹452K
    Projected cash position: ₹87676K
    No liquidity crisis expected
    Total 26-week impact: ₹11759K

 Mild Recession: Economic slowdown scenario
    Weekly net cash flow: ₹85K
    Projected cash position: ₹78117K
    No liquidity crisis expected
    Total 26-week impact: ₹2200K

 Severe Recession: Severe economic downturn
    Weekly net cash flow: ₹-348K
    Projected cash position: ₹66867K
    No liquidity crisis expected
    Total 26-week impact: ₹-9050K

 Supply Chain Crisis: Supply chain disruption scenario
    Weekly net cash flow: ₹87K
    Projected cash position: ₹78190K
    No liquidity crisis expected
    Total 26-week impact: ₹2273K

 Customer Credit Cri

In [ ]:

print("\n PHASE 6: EXECUTIVE SUMMARY & RECOMMENDATIONS")
print("="*80)

print("\n KEY FINDINGS & EXECUTIVE SUMMARY:")

print(f" FINANCIAL HEALTH ASSESSMENT: EXCELLENT")
print(f"    Current liquidity position: ₹{current_cash_position:.0f}K")
print(f"    4-year cash growth: {((current_cash_position / 5000) - 1) * 100:.0f}%")
print(f"    Average annual revenue growth: 4.8%")
print(f"    Liquidity crisis probability: 0.0% (next 6 months)")

print(f"\n CASH FLOW CHARACTERISTICS:")
print(f"    Average weekly net cash flow: ₹{weekly_df['Net_Cash_Flow'].mean():.0f}K")
print(f"    Cash flow volatility: ₹{weekly_df['Net_Cash_Flow'].std():.0f}K")
print(f"    Seasonal peak: Q4 (+₹435K weekly average)")
print(f"    Seasonal low: Q3 (₹191K weekly average)")
print(f"    Negative cash flow weeks: {negative_flow_weeks} out of {len(weekly_df)} ({negative_flow_weeks/len(weekly_df)*100:.1f}%)")

print(f"\n FORECASTING MODEL PERFORMANCE:")
print(f"    Best model: Random Forest Regression")
print(f"    Mean Absolute Error: ₹182K")
print(f"    Root Mean Squared Error: ₹215K")
print(f"    Key predictors: 2-week and 4-week lagged cash flows")

print(f"\n  RISK ASSESSMENT:")
print(f"    Collection risk score: {ar_df['Collection_Risk_Score'].mean():.2f}% (LOW)")
print(f"    Days Sales Outstanding: 12.6 days (EXCELLENT)")
print(f"    Most critical scenario: Customer Credit Crisis")
print(f"    Time to liquidity crisis (worst case): 182.6 weeks")

print(f"\n" + "="*80)
print(" STRATEGIC RECOMMENDATIONS")
print("="*80)

recommendations = [
    {
        "category": " LIQUIDITY MANAGEMENT",
        "items": [
            "Maintain minimum cash buffer of ₹5,000K (current: ₹75,917K )",
            "Establish revolving credit facility for seasonal fluctuations",
            "Consider higher-yield investment options for excess cash",
            "Monitor weekly cash flow forecasts with ±₹215K accuracy range"
        ]
    },
    {
        "category": " OPERATIONAL OPTIMIZATION",
        "items": [
            "Leverage Q4 seasonal strength to build cash reserves",
            "Implement cost management during Q3 low season",
            "Focus on customer relationship management (low collection risk)",
            "Optimize payment terms with suppliers during strong periods"
        ]
    },
    {
        "category": " RISK MITIGATION",
        "items": [
            "Develop contingency plans for revenue declines >15%",
            "Establish early warning system for collection risk >3.5%",
            "Create supplier payment flexibility agreements",
            "Monitor economic indicators: GDP growth and manufacturing PMI"
        ]
    },
    {
        "category": " FORECASTING & MONITORING",
        "items": [
            "Update forecasting model monthly with new data",
            "Implement automated stress testing for scenario planning",
            "Track key leading indicators: 2-week lagged cash flows",
            "Establish quarterly model validation and recalibration"
        ]
    }
]

for rec in recommendations:
    print(f"\n{rec['category']}:")
    for i, item in enumerate(rec['items'], 1):
        print(f"   {i}. {item}")

# Create action items with priorities
print(f"\n" + "="*80)
print(" IMMEDIATE ACTION ITEMS (Next 30 Days)")
print("="*80)

action_items = [
    ("HIGH", "Set up automated weekly cash flow monitoring dashboard"),
    ("HIGH", "Establish credit facility for seasonal working capital needs"),
    ("MEDIUM", "Implement monthly forecasting model updates"),
    ("MEDIUM", "Create stress testing scenarios for board reporting"),
    ("LOW", "Evaluate investment options for excess cash position")
]

for priority, action in action_items:
    print(f"• [{priority:6}] {action}")


print(" MODEL DEPLOYMENT GUIDELINES")


print(f"""
 DEPLOYMENT CHECKLIST:

1. DATA PIPELINE SETUP:
    Weekly data collection from accounting systems
    Automated data validation and quality checks
    Integration with economic indicator feeds

2. MODEL MONITORING:
    Track forecast accuracy against actuals
    Retrain models quarterly or when MAE > ₹250K
    Validate seasonal patterns annually

3. REPORTING FRAMEWORK:
    Weekly: Cash position and short-term forecasts
    Monthly: Model performance and risk assessment
    Quarterly: Stress testing and strategic recommendations

4. STAKEHOLDER COMMUNICATION:
    CFO: Weekly cash position reports
    Board: Quarterly stress testing results
    Treasury: Daily liquidity monitoring
    Operations: Seasonal planning guidance
""")

print(f" PROJECT COMPLETE - CASH FLOW FORECASTING MODEL READY FOR DEPLOYMENT")

# Save key results for future reference
results_summary = {
    'current_cash_position': current_cash_position,
    'forecast_accuracy_mae': 182,
    'forecast_accuracy_rmse': 215,
    'average_weekly_cash_flow': weekly_df['Net_Cash_Flow'].mean(),
    'cash_flow_volatility': weekly_df['Net_Cash_Flow'].std(),
    'collection_risk_score': ar_df['Collection_Risk_Score'].mean(),
    'days_sales_outstanding': risk_df['DSO'].mean(),
    'liquidity_crisis_probability': 0.0,
    'model_deployment_date': pd.Timestamp.now().strftime('%Y-%m-%d')
}

print(f"\n Results saved for model tracking and validation")
print(f"   Deployment date: {results_summary['model_deployment_date']}")
print(f"   Next model review: {(pd.Timestamp.now() + pd.DateOffset(months=3)).strftime('%Y-%m-%d')}")


 PHASE 6: EXECUTIVE SUMMARY & RECOMMENDATIONS

 KEY FINDINGS & EXECUTIVE SUMMARY:
 FINANCIAL HEALTH ASSESSMENT: EXCELLENT
    Current liquidity position: ₹75917K
    4-year cash growth: 1418%
    Average annual revenue growth: 4.8%
    Liquidity crisis probability: 0.0% (next 6 months)

 CASH FLOW CHARACTERISTICS:
    Average weekly net cash flow: ₹339K
    Cash flow volatility: ₹260K
    Seasonal peak: Q4 (+₹435K weekly average)
    Seasonal low: Q3 (₹191K weekly average)
    Negative cash flow weeks: 16 out of 209 (7.7%)

 FORECASTING MODEL PERFORMANCE:
    Best model: Random Forest Regression
    Mean Absolute Error: ₹182K
    Root Mean Squared Error: ₹215K
    Key predictors: 2-week and 4-week lagged cash flows

  RISK ASSESSMENT:
    Collection risk score: 2.33% (LOW)
    Days Sales Outstanding: 12.6 days (EXCELLENT)
    Most critical scenario: Customer Credit Crisis
    Time to liquidity crisis (worst case): 182.6 weeks

 STRATEGIC RECOMMENDATIONS

 LIQUIDITY MANAGEMENT:
   1. M

In [ ]:
display(ar_df.head())

,Date,Total_AR,Current_0_30_Days,Days_31_60,Days_61_90,Days_Over_90,Collection_Risk_Score,Week,Month,Quarter,Year
0,2021-01-04,2384.21,1708.04,367.76,235.65,72.75,1.90,1,1,1,2021
1,2021-01-11,2420.05,1587.21,505.11,191.35,136.38,2.48,2,1,1,2021
2,2021-01-18,1611.74,1043.73,317.91,173.40,76.70,2.50,3,1,1,2021
3,2021-01-25,2266.09,1479.53,451.37,244.40,90.79,2.28,4,1,1,2021
4,2021-02-01,1599.95,1020.94,360.72,175.82,42.46,1.90,5,2,1,2021
